In [207]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import pickle

In [193]:
axis_df = pd.read_csv('Axis.csv')
hdfc_df = pd.read_csv('HDFC.csv')
sbi_df = pd.read_csv('State.csv')
ind_df = pd.read_csv('IndusInd.csv')
icici_df = pd.read_csv('ICICI.csv')

print('axis length',len(axis_df),'hdfc length',len(hdfc_df),'sbi length',len(sbi_df),'IndusInd',len(ind_df),'ICICI length',len(icici_df))
df = [axis_df, hdfc_df,sbi_df,ind_df,icici_df]
result = pd.concat(df)
print(len(result))
result.head()

axis length 290 hdfc length 407 sbi length 598 IndusInd 197 ICICI length 409
1901


,label,news
0,1.0,retain Hold Axis Bank TP revised lower Rs520. ...
1,1.0,"Rajat Bose rajatkbose.com told CNBC-TV18, ""I o..."
2,1.0,India's third largest private sector bank toda...
3,0.0,Axis bank's earnings Rs7.0bn (PLe: Rs9.9bn) lo...
4,1.0,"Nifty closed 9,108 largely remained sideways w..."


In [194]:
def integer(x):
    return int(x)

In [195]:
result = result.sample(frac=1).reset_index(drop=True)
result['label']=result['label'].apply(integer)
print(result.head())

   label                                               news
0      1  Vijay Chopra enochventures.com told CNBC-TV18,...
1      0  bank reported steady asset quality sequential ...
2      0  Monday said expand retail network adding 450 n...
3      1  Mitessh Thakkar mitesshthakkar.com told CNBC-T...
4      1  Hyundai Motor India Monday signed agreement pr...


In [196]:
# grouping by label
print(result.groupby(by='label').count())
print('percentage positive news',(870/len(result))*100 )
print('percentage negative news',(1031/len(result))*100)

       news
label      
0      1031
1       870
percentage positive news 45.765386638611254
percentage negative news 54.23461336138874


In [197]:
len(result.index)
result['news'].iloc[0]

'Vijay Chopra enochventures.com told CNBC-TV18, "With longer term perspective best placed bank (SBI), doubt it. might correct bit, also reality. However, somebody holds longer term perspective, present turn, stock might go Rs 310 probably might see kind correction bought lower prices." Rajat Bose rajatkbose.com told CNBC-TV18, "In (SBI) Rs 305-325 strong resistance zone. manages get past that, definitely upside likely. look SBI chart look chart, go surrender almost 100 percent gains, sometimes even more. kind volatility may great thing even long term investor. So, perspective, would probably bought selling SBI."At 15:16 hrs State Bank India quoting Rs 280.35, Rs 2.55, 0.90 percent BSE.'

In [201]:
#creating a text cleaning function to pass into countvectorizer
def text_remove_punctuations(news):
    """
    The function text_remove_punctuation:
    1. Removes remaining commas
    """
     # Check characters to see if they are in punctuation
    nopunc = [char for char in news if char not in string.punctuation]
      
    
    
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
  

    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]


from sklearn.feature_extraction.text import CountVectorizer
# Vectorization of words, Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_remove_punctuations).fit(result['news'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

#Tokenization or vectorization of all the generated news
news_bow = bow_transformer.transform(result['news'])

print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

#calculating sparsity
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(sparsity))

44893
Shape of Sparse Matrix:  (1901, 44893)
Amount of Non-Zero occurences:  361680
sparsity: 0.42380272887570775


In [202]:
# used for checking the vectorization
news_0 = result['news'][0]
trans_words = bow_transformer.transform([news_0])
#print(trans_words)
#print(trans_words.shape)
bow_transformer.get_feature_names()[44314]

'yoy'

In [203]:
print('sparsity: {}'.format(sparsity))

sparsity: 0.42380272887570775


In [204]:
# creating train and test data:
from sklearn.model_selection import train_test_split

news_train, news_test, label_train, label_test = train_test_split(result['news'], result['label'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

1330 571 1901


In [205]:
# making a pipeline of the above procedure
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=identity)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [209]:
pipeline.fit(news_train,label_train)
filename = 'Model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

In [210]:
#checking the predictions
predictions = pipeline.predict(news_test)
print(predictions[0:10])
print(label_test[0:10])

[0 0 0 0 0 0 0 0 0 0]
873     1
189     0
281     1
170     1
1564    0
1728    0
614     0
8       0
826     1
980     1
Name: label, dtype: int64


In [211]:
# checking the classification report
from sklearn.metrics import classification_report
print(classification_report(predictions,label_test))

              precision    recall  f1-score   support

           0       1.00      0.51      0.68       571
           1       0.00      0.00      0.00         0

    accuracy                           0.51       571
   macro avg       0.50      0.25      0.34       571
weighted avg       1.00      0.51      0.68       571



/home/vasanth/anaconda3/envs/DS/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
